In [1]:
!head -n 10 data/installments_payments.csv

SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
1054186,161674,1.0,6,-1180.0,-1187.0,6948.36,6948.36
1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2085231,193053,2.0,1,-63.0,-63.0,25425.0,25425.0
2452527,199697,1.0,3,-2418.0,-2426.0,24350.13,24350.13
2714724,167756,1.0,2,-1383.0,-1366.0,2165.04,2160.585
1137312,164489,1.0,12,-1384.0,-1417.0,5970.375,5970.375
2234264,184693,4.0,11,-349.0,-352.0,29432.295,29432.295
1818599,111420,2.0,4,-968.0,-994.0,17862.165,17862.165
2723183,112102,0.0,14,-197.0,-197.0,70.74,70.74


In [3]:
import pandas as pd
filename = 'data/application_train.csv'
df = pd.read_csv(filename)
df.head(20) 

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.000,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.000,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.000,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.000,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.000,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
5,100008,0,Cash loans,M,N,Y,0,99000.000,490495.5,27517.5,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
6,100009,0,Cash loans,F,Y,Y,1,171000.000,1560726.0,41301.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,2.0
7,100010,0,Cash loans,M,Y,Y,0,360000.000,1530000.0,42075.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
8,100011,0,Cash loans,F,N,Y,0,112500.000,1019610.0,33826.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
9,100012,0,Revolving loans,M,N,Y,0,135000.000,405000.0,20250.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print(df.dtypes)

SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
FLAG_OWN_REALTY                object
CNT_CHILDREN                    int64
AMT_INCOME_TOTAL              float64
AMT_CREDIT                    float64
AMT_ANNUITY                   float64
AMT_GOODS_PRICE               float64
NAME_TYPE_SUITE                object
NAME_INCOME_TYPE               object
NAME_EDUCATION_TYPE            object
NAME_FAMILY_STATUS             object
NAME_HOUSING_TYPE              object
REGION_POPULATION_RELATIVE    float64
DAYS_BIRTH                      int64
DAYS_EMPLOYED                   int64
DAYS_REGISTRATION             float64
DAYS_ID_PUBLISH                 int64
OWN_CAR_AGE                   float64
FLAG_MOBIL                      int64
FLAG_EMP_PHONE                  int64
FLAG_WORK_PHONE                 int64
FLAG_CONT_MOBILE                int64
FLAG_PHONE  

In [5]:
df[['SK_ID_CURR', 'TARGET']].groupby('TARGET').count()

,SK_ID_CURR
TARGET,
0,282686
1,24825


In [6]:
df[['SK_ID_CURR', 'TARGET']].groupby('TARGET').count()/df.shape[0]

,SK_ID_CURR
TARGET,
0,0.919271
1,0.080729


In [13]:
filename = 'data/HomeCredit_columns_description.csv'
desc_df = pd.read_csv(filename, encoding = "ISO-8859-1")
desc_df[desc_df['Table'] == 'application_{train|test}.csv']

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
5,8,application_{train|test}.csv,FLAG_OWN_REALTY,Flag if client owns a house or flat,NaN
6,9,application_{train|test}.csv,CNT_CHILDREN,Number of children the client has,NaN
7,10,application_{train|test}.csv,AMT_INCOME_TOTAL,Income of the client,NaN
8,11,application_{train|test}.csv,AMT_CREDIT,Credit amount of the loan,NaN
9,12,application_{train|test}.csv,AMT_ANNUITY,Loan annuity,NaN


In [8]:
def get_target_dist(df):
    rows = df.shape[0]
    target_dist_df = df[['SK_ID_CURR', 'TARGET']].groupby('TARGET').count()
    target_dist_df['PERCENT'] = target_dist_df['SK_ID_CURR']*100/rows
    return target_dist_df

In [14]:
filename = 'data/application_train.csv'
train_df = pd.read_csv(filename)
rows = train_df.shape[0]
print(f'total rows: {rows}')
print(get_target_dist(train_df))

total rows: 307511
        SK_ID_CURR    PERCENT
TARGET                       
0           282686  91.927118
1            24825   8.072882


In [15]:
train_df = pd.read_csv(filename, nrows=10000)
rows = train_df.shape[0]
print(f'total rows: {rows}')

total rows: 10000


In [16]:
print(get_target_dist(train_df))

        SK_ID_CURR  PERCENT
TARGET                     
0             9225    92.25
1              775     7.75


In [17]:
train_df = pd.read_csv(filename)
train_df = train_df.sample(n=10000)
rows = train_df.shape[0]
print(f'total rows: {rows}') 
print(get_target_dist(train_df))

total rows: 10000
        SK_ID_CURR  PERCENT
TARGET                     
0             9207    92.07
1              793     7.93


In [18]:
nan_cols = train_df.columns[train_df.isnull().any()]
nan_cnt = train_df[nan_cols].isnull().sum()
print(nan_cnt)

AMT_GOODS_PRICE                    7
NAME_TYPE_SUITE                   38
OWN_CAR_AGE                     6619
OCCUPATION_TYPE                 3120
EXT_SOURCE_1                    5637
EXT_SOURCE_2                      29
EXT_SOURCE_3                    1986
APARTMENTS_AVG                  5083
BASEMENTAREA_AVG                5844
YEARS_BEGINEXPLUATATION_AVG     4887
YEARS_BUILD_AVG                 6641
COMMONAREA_AVG                  6993
ELEVATORS_AVG                   5371
ENTRANCES_AVG                   5037
FLOORSMAX_AVG                   4973
FLOORSMIN_AVG                   6776
LANDAREA_AVG                    5915
LIVINGAPARTMENTS_AVG            6838
LIVINGAREA_AVG                  5024
NONLIVINGAPARTMENTS_AVG         6945
NONLIVINGAREA_AVG               5504
APARTMENTS_MODE                 5083
BASEMENTAREA_MODE               5844
YEARS_BEGINEXPLUATATION_MODE    4887
YEARS_BUILD_MODE                6641
COMMONAREA_MODE                 6993
ELEVATORS_MODE                  5371
E

In [19]:
import numpy as np

y = []
data = []

target_col = 'TARGET'
features = list([x for x in train_df.columns if x != target_col])

for row in train_df.to_dict('records'):
    y.append(row[target_col])
    data.append({k: row[k] for k in features})
    
y = np.array(y)

In [21]:
from sklearn.model_selection import train_test_split

data_train, data_val, y_train, y_val = train_test_split(data, y, train_size=0.8, stratify=y)
print(f'data_train cnt: {len(data_train)}')
print(f'data_val cnt: {len(data_val)}')

data_train cnt: 8000
data_val cnt: 2000


/Users/churanchen/Coding/home-credit-kaggle/wcd_venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [22]:
from collections import defaultdict

def get_y_dist(y):
    target2cnt = defaultdict(int)
    for yi in y:
        target2cnt[yi] += 1
    
    print('target\tcnt\tratio')
    for target in sorted(target2cnt):
        cnt = target2cnt[target]
        print(f'{target}\t{cnt}\t{cnt/len(y)}')

In [23]:
print('target distribution in training data')
get_y_dist(y_train)

print('\ntarget distribution in validation data')
get_y_dist(y_val)

target distribution in training data
target	cnt	ratio
0	7366	0.92075
1	634	0.07925

target distribution in validation data
target	cnt	ratio
0	1841	0.9205
1	159	0.0795


In [24]:
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()
X_train = vectorizer.fit_transform(data_train)
print(f'after vectorization: {X_train.shape}')

after vectorization: (8000, 244)


In [25]:
for i, feature in enumerate(vectorizer.feature_names_):
    print(f'{i}\t{feature}')

0	AMT_ANNUITY
1	AMT_CREDIT
2	AMT_GOODS_PRICE
3	AMT_INCOME_TOTAL
4	AMT_REQ_CREDIT_BUREAU_DAY
5	AMT_REQ_CREDIT_BUREAU_HOUR
6	AMT_REQ_CREDIT_BUREAU_MON
7	AMT_REQ_CREDIT_BUREAU_QRT
8	AMT_REQ_CREDIT_BUREAU_WEEK
9	AMT_REQ_CREDIT_BUREAU_YEAR
10	APARTMENTS_AVG
11	APARTMENTS_MEDI
12	APARTMENTS_MODE
13	BASEMENTAREA_AVG
14	BASEMENTAREA_MEDI
15	BASEMENTAREA_MODE
16	CNT_CHILDREN
17	CNT_FAM_MEMBERS
18	CODE_GENDER=F
19	CODE_GENDER=M
20	COMMONAREA_AVG
21	COMMONAREA_MEDI
22	COMMONAREA_MODE
23	DAYS_BIRTH
24	DAYS_EMPLOYED
25	DAYS_ID_PUBLISH
26	DAYS_LAST_PHONE_CHANGE
27	DAYS_REGISTRATION
28	DEF_30_CNT_SOCIAL_CIRCLE
29	DEF_60_CNT_SOCIAL_CIRCLE
30	ELEVATORS_AVG
31	ELEVATORS_MEDI
32	ELEVATORS_MODE
33	EMERGENCYSTATE_MODE
34	EMERGENCYSTATE_MODE=No
35	EMERGENCYSTATE_MODE=Yes
36	ENTRANCES_AVG
37	ENTRANCES_MEDI
38	ENTRANCES_MODE
39	EXT_SOURCE_1
40	EXT_SOURCE_2
41	EXT_SOURCE_3
42	FLAG_CONT_MOBILE
43	FLAG_DOCUMENT_10
44	FLAG_DOCUMENT_11
45	FLAG_DOCUMENT_12
46	FLAG_DOCUMENT_13
47	FLAG_DOCUMENT_14
48	FLAG_DOCUMENT_15

In [26]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MaxAbsScaler

imputer = Imputer(strategy='mean')
X_train = imputer.fit_transform(X_train)

scaler = MaxAbsScaler()
X_train = scaler.fit_transform(X_train.toarray())
print(f'X_train data type: {type(X_train)}')
print(f'X_train: {X_train.shape})')

X_train data type: <class 'numpy.ndarray'>
X_train: (8000, 244))


In [27]:
X_val = vectorizer.transform(data_val)
X_val = imputer.transform(X_val)
X_val = scaler.transform(X_val)
print(f'X_val data type: {type(X_val)}')
print(f'X_val: {X_val.shape})')

X_val data type: <class 'scipy.sparse.csc.csc_matrix'>
X_val: (2000, 244))


In [28]:
# Model Training
from sklearn.linear_model import LogisticRegression
import time

model = LogisticRegression(class_weight='balanced')

start = time.time()
print(f'Fitting model on {X_train.shape[0]} samples...')
model.fit(X_train, y_train)

end = time.time()
print('Finished model training in %.3f seconds.' % (end-start))

Fitting model on 8000 samples...
Finished model training in 0.521 seconds.


In [29]:
def get_sample_weights(y):
    weights = []
    for yi in y:
        weights.append(10 if yi else 1)
    return np.array(weights)

from sklearn.linear_model import LogisticRegression
import time

model = LogisticRegression()

start = time.time()
print(f'Fitting model on {X_train.shape[0]} samples...')
model.fit(X_train, y_train, sample_weight=get_sample_weights(y_train))

end = time.time()
print('Finished model training in %.3f seconds.' % (end-start))

Fitting model on 8000 samples...
Finished model training in 0.627 seconds.


In [30]:
y_preds = model.predict(X_val)

In [31]:
for i, y_pred in enumerate(y_preds):
    y_true = y_val[i]
    print(f'i\ty_pred: {y_pred}, y_true: {y_true}')

i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 1
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 1
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 1
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0

i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 1
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 1, y_true: 1
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 1
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 1
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 0, y_true: 0
i	y_pred: 1, y_true: 0
i	y_pred: 1, y_true: 1
i	y_pred: 1

In [32]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_val, y_pred=y_preds, labels=[0, 1], target_names=['NO', 'YES']))

             precision    recall  f1-score   support

         NO       0.96      0.75      0.84      1841
        YES       0.17      0.60      0.26       159

avg / total       0.89      0.73      0.79      2000



In [33]:
def evaluate(X_val, y_val):
    from sklearn.metrics import roc_curve, roc_auc_score, auc
    pos_idx = list(model.classes_).index(1)

    print(X_val.shape, model.predict_proba(X_val).shape, pos_idx)
    y_score = model.predict_proba(X_val)[:,pos_idx]
    print(y_score.shape)
    fpr, tpr, _ = roc_curve(y_val, y_score, pos_label=1)
    roc_auc = roc_auc_score(y_val, y_score)
    
    return roc_auc, fpr, tpr

In [34]:
print(X_val.shape)
print(y_val.shape)
roc_auc, fpr, tpr = evaluate(X_val, y_val)

(2000, 244)
(2000,)
(2000, 244) (2000, 2) 1
(2000,)


In [35]:
import matplotlib.pyplot as plt
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")

In [38]:
prev_app_filename = 'data/previous_application.csv'
prev_app_df = pd.read_csv(prev_app_filename)
prev_app_df.head(20)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
5,1383531,199383,Cash loans,23703.930,315000.0,340573.5,NaN,315000.0,SATURDAY,8,...,XNA,18.0,low_normal,Cash X-Sell: low,365243.0,-654.0,-144.0,-144.0,-137.0,1.0
6,2315218,175704,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,11,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
7,1656711,296299,Cash loans,NaN,0.0,0.0,NaN,NaN,MONDAY,7,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
8,2367563,342292,Cash loans,NaN,0.0,0.0,NaN,NaN,MONDAY,15,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
9,2579447,334349,Cash loans,NaN,0.0,0.0,NaN,NaN,SATURDAY,15,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
filename = 'data/HomeCredit_columns_description.csv'
desc_df = pd.read_csv(filename, encoding = "ISO-8859-1")
desc_df[desc_df['Table'] == 'previous_application.csv']

,Unnamed: 0,Table,Row,Description,Special
173,176,previous_application.csv,SK_ID_PREV,ID of previous credit in Home credit related t...,hashed
174,177,previous_application.csv,SK_ID_CURR,ID of loan in our sample,hashed
175,178,previous_application.csv,NAME_CONTRACT_TYPE,"Contract product type (Cash loan, consumer loa...",NaN
176,179,previous_application.csv,AMT_ANNUITY,Annuity of previous application,NaN
177,180,previous_application.csv,AMT_APPLICATION,For how much credit did client ask on the prev...,NaN
178,181,previous_application.csv,AMT_CREDIT,Final credit amount on the previous applicatio...,NaN
179,182,previous_application.csv,AMT_DOWN_PAYMENT,Down payment on the previous application,NaN
180,183,previous_application.csv,AMT_GOODS_PRICE,Goods price of good that client asked for (if ...,NaN
181,184,previous_application.csv,WEEKDAY_APPR_PROCESS_START,On which day of the week did the client apply ...,NaN
182,185,previous_application.csv,HOUR_APPR_PROCESS_START,Approximately at what day hour did the client ...,rounded


In [41]:
prev_agg = prev_app_df.groupby('SK_ID_CURR')
prev_df = prev_agg.agg({'SK_ID_PREV': 'count', 'AMT_ANNUITY': 'sum'}).rename(columns={
    'SK_ID_PREV': 'PREV_APPS', 'AMT_ANNUITY': 'PREV_AMT_ANNUITY'})

In [42]:
prev_df.head(20)

,PREV_APPS,PREV_AMT_ANNUITY
SK_ID_CURR,,
100001,1,3951.000
100002,1,9251.775
100003,3,169661.970
100004,1,5357.250
100005,2,4813.200
100006,9,141907.050
100007,6,73672.830
100008,5,63358.785
100009,7,70359.885


In [44]:
curr_prev_df = train_df.fillna(value=train_df.mean()).join(prev_df, on='SK_ID_CURR', how='left')
curr_prev_df[['PREV_APPS', 'PREV_AMT_ANNUITY']] = curr_prev_df[['PREV_APPS', 'PREV_AMT_ANNUITY']].fillna(value=0)
print(curr_prev_df.shape[0])
curr_prev_df

10000


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,PREV_APPS,PREV_AMT_ANNUITY
223674,359072,0,Cash loans,F,N,Y,0,54000.0,112500.0,4765.5,...,0,0,0.007174,0.006364,0.032631,0.267646,0.260819,1.906387,2.0,6983.730
297040,444140,0,Cash loans,F,N,N,0,166500.0,450000.0,16965.0,...,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,13280.625
263861,405519,0,Cash loans,F,N,Y,1,122400.0,545040.0,20677.5,...,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,16040.925
203732,336191,0,Cash loans,F,Y,Y,0,675000.0,722394.0,19183.5,...,0,0,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,6.0,58086.045
70375,181638,0,Cash loans,F,N,N,0,153000.0,270000.0,10309.5,...,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,6976.620
276534,420462,0,Cash loans,F,N,Y,2,108000.0,253737.0,27454.5,...,0,0,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,4.0,30311.100
30042,134875,0,Cash loans,F,N,Y,0,112500.0,1312110.0,55723.5,...,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,8119.305
229967,366362,0,Cash loans,F,N,Y,0,36000.0,54000.0,5859.0,...,0,0,0.007174,0.006364,0.032631,0.267646,0.260819,1.906387,2.0,9328.140
15375,117945,1,Cash loans,F,N,Y,0,112500.0,840951.0,33480.0,...,0,0,0.007174,0.006364,0.032631,0.267646,0.260819,1.906387,4.0,56738.160
239325,377170,0,Revolving loans,F,Y,Y,2,135000.0,450000.0,22500.0,...,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.0,31542.885


In [45]:
filename = 'data/bureau.csv'
bureau_df = pd.read_csv(filename)
active_bureau_df = bureau_df[bureau_df['CREDIT_ACTIVE']=='Active']
active_bureau_df

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.000,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.00,71017.380,108982.620,0.0,Credit card,-31,NaN
6,215354,5714468,Active,currency 1,-43,0,79.0,NaN,0.0,0,42103.80,42103.800,0.000,0.0,Consumer credit,-22,NaN
9,162297,5714471,Active,currency 1,-1146,0,-484.0,NaN,0.0,0,4500.00,0.000,0.000,0.0,Credit card,-690,NaN
10,162297,5714472,Active,currency 1,-1146,0,-180.0,NaN,0.0,0,337500.00,0.000,0.000,0.0,Credit card,-690,NaN
12,162297,5714474,Active,currency 1,-277,0,5261.0,NaN,0.0,0,7033500.00,NaN,NaN,0.0,Mortgage,-31,NaN
13,402440,5714475,Active,currency 1,-96,0,269.0,NaN,0.0,0,89910.00,76905.000,0.000,0.0,Consumer credit,-22,NaN


In [46]:
active_bureau_agg = active_bureau_df.groupby('SK_ID_CURR')
active_bureau_agg_df = active_bureau_agg.agg({'AMT_CREDIT_SUM_DEBT': 'sum', 'CNT_CREDIT_PROLONG': 'sum'})
active_bureau_agg_df

,AMT_CREDIT_SUM_DEBT,CNT_CREDIT_PROLONG
SK_ID_CURR,,
100001,596686.500,0
100002,245781.000,0
100003,0.000,0
100005,568408.500,0
100008,240057.000,0
100009,1077349.500,0
100010,348007.500,0
100014,758214.000,0
100016,63724.500,0


In [47]:
df = curr_prev_df.join(active_bureau_agg_df, on='SK_ID_CURR', how='left')
df[['AMT_CREDIT_SUM_DEBT', 'CNT_CREDIT_PROLONG']] = df[['AMT_CREDIT_SUM_DEBT', 'CNT_CREDIT_PROLONG']].fillna(value=0)
print(df.shape[0])
df.shape

10000


(10000, 126)

In [48]:
df

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,PREV_APPS,PREV_AMT_ANNUITY,AMT_CREDIT_SUM_DEBT,CNT_CREDIT_PROLONG
223674,359072,0,Cash loans,F,N,Y,0,54000.0,112500.0,4765.5,...,0.007174,0.006364,0.032631,0.267646,0.260819,1.906387,2.0,6983.730,0.000000e+00,0.0
297040,444140,0,Cash loans,F,N,N,0,166500.0,450000.0,16965.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,13280.625,1.140750e+04,0.0
263861,405519,0,Cash loans,F,N,Y,1,122400.0,545040.0,20677.5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,16040.925,0.000000e+00,0.0
203732,336191,0,Cash loans,F,Y,Y,0,675000.0,722394.0,19183.5,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,6.0,58086.045,3.279884e+06,0.0
70375,181638,0,Cash loans,F,N,N,0,153000.0,270000.0,10309.5,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,6976.620,4.820445e+05,0.0
276534,420462,0,Cash loans,F,N,Y,2,108000.0,253737.0,27454.5,...,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,4.0,30311.100,1.551353e+07,0.0
30042,134875,0,Cash loans,F,N,Y,0,112500.0,1312110.0,55723.5,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,8119.305,0.000000e+00,0.0
229967,366362,0,Cash loans,F,N,Y,0,36000.0,54000.0,5859.0,...,0.007174,0.006364,0.032631,0.267646,0.260819,1.906387,2.0,9328.140,0.000000e+00,0.0
15375,117945,1,Cash loans,F,N,Y,0,112500.0,840951.0,33480.0,...,0.007174,0.006364,0.032631,0.267646,0.260819,1.906387,4.0,56738.160,0.000000e+00,0.0
239325,377170,0,Revolving loans,F,Y,Y,2,135000.0,450000.0,22500.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.0,31542.885,5.989950e+04,0.0


In [49]:
import pickle
from sklearn.externals import joblib

joblib.dump(model, 'model')
joblib.dump(scaler, 'scaler')
joblib.dump(vectorizer, 'vectorizer')

model = joblib.load('model')
scaler = joblib.load('scaler')
vectorizer = joblib.load('vectorizer')